In [2]:
#Files are provided as event logs
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.importer.xes import importer as xes_importer

### Preprocessing

In [3]:
import pm4py

process_type = "NONFFF"

log = xes_importer.apply("../../data/01_WM/01_raw/20230411/unique case_names/xes_" + process_type + "_unique.xml")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/01_WM/01_raw/20230411/unique case_names/xes_NONFFF_unique.xml'

In [1]:
import preprocessing as pp

log = pp.modify_timestamps(log)
log = pp.drop_events(log)
log = pp.adjust_concept_names(log)
log = pp.refine_activity_classifier(log)

if(process_type == "FFF"):
    log = pp.assign_gates_FFF(log)
elif(process_type == "EC"):
    log = pp.assign_gates_EC(log)
else:
    log = pp.assign_gates_NONFFF(log)




NameError: name 'log' is not defined

In [4]:
log = pp.remove_incomplete_traces(log, process_type)

In [6]:
print(len(log))

530


In [1]:
xes_exporter.apply(log, '../../data/01_WM/03_final/Preprocessed_ModificationProcess_EventLog/ProdCompleted' + process_type + '_Preprocessed.xes')

NameError: name 'xes_exporter' is not defined

### Train Test Split

In [ ]:
import split
log = xes_importer.apply('../../data/01_WM/03_final/Preprocessed_ModificationProcess_EventLog/ProdCompleted' + process_type + '_Preprocessed.xes')

split.time_based_split(log, '../../data/01_WM/03_final/TrainTestSplit', process_type)

### Feature Extraction

In [ ]:
import feature_extraction as fe

def feature_extraction(log, log_type="Train"):
    log = fe.feature_processed_documents(log, process_type)
    log = fe.feature_number_of_parts(log, process_type)
    log = fe.feature_number_of_initiated_workflows(log, process_type)
    log = fe.feature_number_of_different_roles(log, process_type)
    log = fe.feature_qma_check_relevance(log)
    log = fe.t0_to_be_changed(log)
    log = fe.t0_delete_parts_documents(log)
    log = fe.t0_start_division(log)
    log = fe.number_tailored_criteria(log, process_type)
    log = fe.number_completed_tasks(log, process_type)
    log = fe.number_added_tasks(log, process_type)
    log = fe.international_involved(log, process_type)
    log = fe.involved_divisions_encoded(log, process_type)
    log = fe.number_divisions(log, process_type)
    log = fe.t0_start_month(log)
    log = fe.feature_last_three_events_per_case(log, process_type)
    log = fe.feature_involved_resources(log, process_type)
    log = fe.feature_multiple_roles(log, process_type)
    log = fe.calculate_lead_time_per_gate(log, process_type)
    log = fe.calculate_total_and_remaining_leadtime(log, process_type)
    
    fe.extract_and_save(log, '../../data/01_WM/03_final/Feature Extraction', log_type, process_type)

log_train = xes_importer.apply('../../data/01_WM/03_final/TrainTestSplit/' + process_type + '_Train.xes')
log_test = xes_importer.apply('../../data/01_WM/03_final/TrainTestSplit/' + process_type + '_Test.xes')

feature_extraction(log_train, "Train")
feature_extraction(log_test, "Test")